In [30]:
import pandas as pd
import numpy as np

# 1. Carregar a base original (Chácara)

In [31]:
# Carrega o arquivo de entrada (arquivo organizado em `filling_Ceps/`)
df = pd.read_csv('../filling_Ceps/Elvira - Chacara - INEP 35107700 Dados ADS_coords_corrigidas_com_enderecos.csv')

# 2. Tratar a renda para número

In [32]:
import re

def parse_renda_seguro(x):
    if pd.isna(x):
        return np.nan
    x = str(x)

    # remove qualquer símbolo que não seja número, vírgula ou ponto
    x = re.sub(r"[^0-9,\.]", "", x)

    # caso venha no formato errado tipo "19.448.421.299.999.900"
    # mantemos somente o último grupo decimal
    if x.count(".") > 1:
        # remove TODOS os pontos; eles NÃO representam milhar
        x = x.replace(".", "")

    # agora troca vírgula por ponto
    x = x.replace(",", ".")

    try:
        return float(x)
    except:
        return np.nan


df["renda_media_num"] = df["renda_media"].apply(parse_renda_seguro)

# 3. Criar Total_0_9 (0–4 + 5–9 anos)

In [33]:
df["Total_0_9"] = (
    df["v01031_0_4anos"].fillna(0) +
    df["v01032_5_9anos"].fillna(0)
)

# 4. Corrigir renda para 2025 (inflação)

In [34]:
inflation_factor = 1.155
df["renda_atualizada_2025"] = df["renda_media_num"] * inflation_factor

# 5. Score linha a linha (informativo, não consolidado)

In [35]:
df["score_trafego_2025"] = df["renda_atualizada_2025"] * df["Total_0_9"]

# 6. Consolidar por CEP (SEM SOMAR — usar média!)

In [36]:
df_cep = (
    df.groupby("CEP", as_index=False)
      .agg({
          "Bairro": lambda x: x.mode().iat[0] if not x.mode().empty else x.iloc[0],
          "renda_atualizada_2025": "median",
          "Total_0_9": "median",
          "populacao_total": "median"
      })
)

In [37]:
# Renomear colunas para refletir que são médias, não totais
df_cep = df_cep.rename(columns={
    "renda_atualizada_2025": "renda_mediana_2025",
    "Total_0_9": "mediana_criancas_0_9",
    "populacao_total": "populacao_mediana"
})

# 7. Score final no nível do CEP (correto)

In [38]:
df_cep["score_trafego_2025"] = (
    df_cep["renda_mediana_2025"] * df_cep["mediana_criancas_0_9"]
)

# 8. Ranking final

In [39]:
top_ceps = df_cep.sort_values("score_trafego_2025", ascending=False)

print(top_ceps.head(15))

           CEP                 Bairro  renda_mediana_2025  \
171  04727-002            Santo Amaro        18074.722050   
100  04662-901     Jardim Santo Amaro        34613.374950   
162  04719-050         Granja Julieta        26215.791525   
154  04717-004  Chácara Santo Antônio        16986.642750   
117  04696-000           Campo Grande        20353.560150   
206  04755-070            Santo Amaro        16676.213400   
274  05708-290           Vila Andrade        25081.824075   
132  04708-001      Jardim Petrópolis        17478.776700   
21   04566-001          Brooklin Novo        34482.871500   
28   04567-110          Brooklin Novo        32052.220200   
173  04730-990         Granja Julieta        18504.491775   
197  04750-000            Santo Amaro        13013.708400   
81   04625-022         Brooklin Velho        22908.183375   
141  04711-130          Brooklin Novo        28064.819475   
212  04766-000                Socorro         6133.084650   

     mediana_criancas_0

In [40]:
# Arredondar para 2 casas decimais (padrão monetário)
top_ceps["renda_mediana_2025"] = top_ceps["renda_mediana_2025"].round(2)
top_ceps["score_trafego_2025"] = top_ceps["score_trafego_2025"].round(2)

In [41]:
# Salvar resultado agregado na pasta do notebook (comportamento original)
top_ceps.to_csv("ElviraBrandao_Chacara_top_ceps_2025.csv", index=False)

In [42]:
# Lista dos bairros desejados
bairros_desejados = ["Granja Julieta", "Chácara Santo Antônio", "Santo Amaro", "Chácara Flora", "Jardim Santo Amaro", "Vila Andrade"]

# Filtrar diretamente no top_ceps
top_ceps_filtrado = top_ceps[top_ceps["Bairro"].isin(bairros_desejados)]

top_ceps_filtrado

,CEP,Bairro,renda_mediana_2025,mediana_criancas_0_9,populacao_mediana,score_trafego_2025
171,04727-002,Santo Amaro,18074.72,117.0,644.0,2114742.48
100,04662-901,Jardim Santo Amaro,34613.37,60.0,546.0,2076802.50
162,04719-050,Granja Julieta,26215.79,73.5,558.5,1926860.68
154,04717-004,Chácara Santo Antônio,16986.64,108.0,680.0,1834557.42
206,04755-070,Santo Amaro,16676.21,92.0,554.0,1534211.63
...,...,...,...,...,...,...
148,04715-002,Santo Amaro,6783.77,8.5,202.0,57662.06
200,04751-000,Jardim Santo Amaro,7331.07,5.5,96.0,40320.87
168,04726-170,Chácara Santo Antônio,12216.54,2.5,264.0,30541.35
146,04714-002,Chácara Santo Antônio,4573.41,4.0,104.0,18293.63


In [43]:
top_ceps_filtrado.to_csv("ElviraBrandao_Chacara_ceps_filtrados_2025.csv", index=False)